In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [4]:
daily_aggregated_df = pd.DataFrame()
temp_df = pd.DataFrame()

year = '2023'

df = pd.read_csv(f'ProcessedCitibikeData/ProcessedCitibikeData/CitibikeProcessed{year}.csv')

df = df[~((df['start_zone_encoded'] == 264) | (df['start_zone_encoded'] == 265) | (df['end_zone_encoded'] == 264) | (df['end_zone_encoded'] == 265))]
df['start_time'] = pd.to_datetime(df['unix_start_time'], unit='s')
df['stop_time'] = pd.to_datetime(df['unix_stop_time'], unit='s')
df = df[['start_zone_encoded', 'end_zone_encoded', 'start_time', 'stop_time']]
df['weight'] = 0

# Find the start time of the first day
start_time = df['start_time'].min().normalize()
t = 0

while start_time <= df['stop_time'].max():
    # Define the end time of the current hour
    end_time = start_time + pd.Timedelta(days=1)
    
    # Slice DataFrame for the current hour
    daily_df = df[(df['start_time'] >= start_time) & (df['start_time'] < end_time)]
    
    # Aggregate data for the current hour
    total_df = daily_df.groupby(['start_zone_encoded', 'end_zone_encoded']).agg({'start_time':'min', 'stop_time':'max', 'weight':'count'}).reset_index()
    total_df['time'] = t
    daily_aggregated_df = pd.concat([daily_aggregated_df, total_df], ignore_index=True)
    
    # Move to the next hour
    start_time += pd.Timedelta(days=1)
    t += 1

# Now hourly_aggregated_df contains aggregated data on an hourly scale starting from 12:00 am of the first day


In [5]:
daily_aggregated_df

,start_zone_encoded,end_zone_encoded,start_time,stop_time,weight,time
0,4,13,2023-01-01 02:16:32,2023-01-01 02:49:23,4,0
1,4,17,2023-01-01 15:17:57,2023-01-01 18:58:29,2,0
2,4,33,2023-01-01 13:37:36,2023-01-01 13:56:08,1,0
3,4,34,2023-01-01 16:50:15,2023-01-01 17:05:29,1,0
4,4,37,2023-01-01 02:25:50,2023-01-01 04:25:06,2,0
...,...,...,...,...,...,...
2343108,263,238,2023-12-31 02:20:42,2023-12-31 23:20:53,10,364
2343109,263,239,2023-12-31 11:21:20,2023-12-31 22:47:14,18,364
2343110,263,244,2023-12-31 09:52:17,2023-12-31 16:16:10,2,364
2343111,263,256,2023-12-31 12:37:49,2023-12-31 13:22:27,1,364


In [7]:
daily_aggregated_df = daily_aggregated_df.drop(['start_time', 'stop_time'], axis=1)

daily_aggregated_df

,start_zone_encoded,end_zone_encoded,weight,time
0,4,13,4,0
1,4,17,2,0
2,4,33,1,0
3,4,34,1,0
4,4,37,2,0
...,...,...,...,...
2343108,263,238,10,364
2343109,263,239,18,364
2343110,263,244,2,364
2343111,263,256,1,364


In [8]:
daily_aggregated_df.to_csv('dailyAggregatedCitibike2023_practice_for_train.csv', index=False)